### Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Model Architecture

In [2]:
# Model Architecture
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 for 7 emotion classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 12800)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [3]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    'fer_2013/train',
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    subset='training',
    batch_size=64
)

validation_generator = datagen.flow_from_directory(
    'fer_2013/test',
    target_size=(48, 48),
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation',
    batch_size=64
)


Found 22968 images belonging to 7 classes.
Found 1432 images belonging to 7 classes.


In [4]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=3
)

# Save the model
model.save('emotion_recognition_model.h5')


Epoch 1/3
359/359 [==============================] - 307s 849ms/step - loss: 1.7286 - accuracy: 0.3045 - val_loss: 1.5839 - val_accuracy: 0.3792
Epoch 2/3
359/359 [==============================] - 61s 170ms/step - loss: 1.5660 - accuracy: 0.3908 - val_loss: 1.4680 - val_accuracy: 0.4469
Epoch 3/3
359/359 [==============================] - 58s 161ms/step - loss: 1.4783 - accuracy: 0.4302 - val_loss: 1.4076 - val_accuracy: 0.4679


In [5]:
import cv2
import numpy as np

# Load pre-trained model and cascade classifier
model = tf.keras.models.load_model('emotion_recognition_model.h5')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Start video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_gray = roi_gray / 255.0
        roi_gray = np.expand_dims(roi_gray, axis=0)
        roi_gray = np.expand_dims(roi_gray, axis=-1)
        
        # Predict emotion
        prediction = model.predict(roi_gray)
        emotion = emotion_labels[np.argmax(prediction)]
        
        # Draw rectangle and text
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    cv2.imshow('Emotion Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 39ms/step
